## Importing stuff

In [1]:
import sys, os
import tsai
from tsai.all import *
display(HTML("<style>.container {width:95% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd
import pickle

In [3]:
sys.path.append(os.path.abspath('../../'))
from utils import utils

## Printing configs

In [4]:
print('tsai       :', tsai.__version__)
print('fastai2    :', fastai2.__version__)
print('fastcore   :', fastcore.__version__)
print('torch      :', torch.__version__)
print('scipy      :', sp.__version__)
print('numpy      :', np.__version__)
print('pandas     :', pd.__version__)
print(f'Total RAM  : {bytes2GB(psutil.virtual_memory().total):5.2f} GB')
print(f'Used RAM   : {bytes2GB(psutil.virtual_memory().used):5.2f} GB')
print('n_cpus     :', cpus)
iscuda = torch.cuda.is_available()
if iscuda: print('device     : {} ({})'.format(device, torch.cuda.get_device_name(0)))
else: print('device     :', device)

tsai       : 0.1.0
fastai2    : 0.0.18
fastcore   : 0.1.18
torch      : 1.5.0
scipy      : 1.4.1
numpy      : 1.18.1
pandas     : 1.0.4
Total RAM  : 62.75 GB
Used RAM   :  2.91 GB
n_cpus     : 16
device     : cuda (GeForce GTX 1080)


## Loading data

In [5]:
path_to_data='../../../'

In [6]:
data=np.load(os.path.abspath(path_to_data+'data-002.npy'),allow_pickle=True)
Y=np.load(path_to_data+'Y.npy',allow_pickle=True)
labels =pd.read_csv(path_to_data+'labels.csv')

train_fold=8
val_fold=9
test_fold=10

# 10th fold for testing (9th for now)
X_test = data[labels.strat_fold == test_fold]
y_test = Y[labels.strat_fold == test_fold]
# 9th fold for validation (8th for now)
X_val = data[labels.strat_fold == val_fold]
y_val = Y[labels.strat_fold == val_fold]
# rest for training
X_train = data[labels.strat_fold <= train_fold]
y_train = Y[labels.strat_fold <= train_fold]

In [7]:
# Preprocess signal data
X_train, X_val, X_test = utils.preprocess_signals(X_train, X_val, X_test,'/content/')
n_classes = y_train.shape[1]
X_train = np.reshape(X_train,[X_train.shape[0],X_train.shape[2],X_train.shape[1]])
X_val = np.reshape(X_val,[X_val.shape[0],X_val.shape[2],X_val.shape[1]])
X_test = np.reshape(X_test,[X_test.shape[0],X_test.shape[2],X_test.shape[1]])

In [8]:
display(X_train.shape)
_, features, seq_len = X_train.shape

(17111, 12, 1000)

In [9]:
X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
X_val = torch.tensor(X_val, dtype=torch.float32, device=device)
labels = np.unique(y_train)
transform = {}
for i, l in enumerate(labels): transform[l] = i
y_train = np.vectorize(transform.get)(y_train)
y_val = np.vectorize(transform.get)(y_val)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

torch.Size([17111, 12, 1000]) torch.Size([2156, 12, 1000]) (17111, 5) (2156, 5)


Checking shapes

In [10]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

(torch.Size([17111, 12, 1000]),
 (17111, 5),
 torch.Size([2156, 12, 1000]),
 (2156, 5))

UCR:(torch.Size([160, 10, 400]), (160,), torch.Size([74, 10, 400]), (74,))

ECG:(torch.Size([17111, 12, 1000]),(17111, 5),torch.Size([2156, 12, 1000]),(2156, 5))

## Instantiating the model

In [11]:
model = InceptionTime(X_train.shape[1],y_train.shape[1]).to(device)

## Training

In [12]:
model(X_train)
# net.zero_grad()
# out.backward(torch.randn(1, 10))
#X_train_tfm = model(X_train).cpu().numpy()
#X_val_tfm = model(X_val).cpu().numpy()

RuntimeError: CUDA out of memory. Tried to allocate 4.08 GiB (GPU 0; 7.93 GiB total capacity; 4.94 GiB already allocated; 2.50 GiB free; 4.95 GiB reserved in total by PyTorch)

## Evaluating

In [ ]:
pred = ridge.predict(X_val_tfm)
pred

In [ ]:
y_val.shape,pred.shape

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
y_val_df = pd.get_dummies(pd.Series(y_val))
y_pred_df = pd.get_dummies(pd.Series(pred))

In [ ]:
df_result = utils.evaluate_experiment(y_val_df.values,y_pred_df.values)
df_result